# 17.2 Zadanie Random Forest

#### Import niezbednych bibliotek i paczek

In [1]:
import numpy as np
import pandas as pd
import string
import nltk
import itertools
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Definicje wszystkich funkcji

In [2]:
def remove_puncation(text):
    cleaned = ''.join([word for word in text if word not in string.punctuation])
    return cleaned
def tokenize(text):
    # Usunięcie wielkich liter
    clean_text = text.lower()
    # Tokenizacja
    tokenized_text = nltk.word_tokenize(clean_text)
    return tokenized_text
def remove_stopwords(text):
    without_stopwords = [word for word in text if word not in stopwords]
    return without_stopwords
def stemming(text):
    stemmed_words = [stemmer.stem(word) for word in text]
    return stemmed_words
def lemmatizing(text):
    lemmatized_words = [lemmater.lemmatize(word) for word in text]
    return lemmatized_words

#### Import pliku z danymi i jego formatowanie

In [3]:
spam_dataset = pd.read_csv('spam.csv', encoding = "ISO-8859-1", usecols=[0, 1], names=['Spam', 'Text'],
                           skiprows=1)
spam_dataset['Spam'] = spam_dataset['Spam'].replace(['ham', 'spam'], [0, 1])
spam_dataset

C:\Users\piotr\AppData\Local\Temp\ipykernel_33660\1996902043.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  spam_dataset['Spam'] = spam_dataset['Spam'].replace(['ham', 'spam'], [0, 1])


,Spam,Text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


#### Obróbka danych - czyszczenie, tokenizacja, usuwanie stop words, Stemming, Lametyzcja

In [4]:
spam_dataset['Cleaned_Text'] = spam_dataset['Text'].apply(lambda x: remove_puncation(x))
spam_dataset['Tokenized_Text'] = spam_dataset['Cleaned_Text'].apply(lambda x: tokenize(x))
stopwords = nltk.corpus.stopwords.words("english")
spam_dataset['WithoutStop_Text'] = spam_dataset['Tokenized_Text'].apply(lambda x: remove_stopwords(x))
stemmer = nltk.PorterStemmer()
spam_dataset['Stemmed_Text'] = spam_dataset['WithoutStop_Text'].apply(lambda x: stemming(x))
lemmater = nltk.WordNetLemmatizer()
spam_dataset['Lemmatized_Text'] = spam_dataset['WithoutStop_Text'].apply(lambda x: lemmatizing(x))
spam_dataset

,Spam,Text,Cleaned_Text,Tokenized_Text,WithoutStop_Text,Stemmed_Text,Lemmatized_Text
0,0,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazi, avail, bugi, n, gre...","[go, jurong, point, crazy, available, bugis, n..."
1,0,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]","[ok, lar, joke, wif, u, oni]","[ok, lar, joking, wif, u, oni]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]","[u, dun, say, earli, hor, u, c, alreadi, say]","[u, dun, say, early, hor, u, c, already, say]"
4,0,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t...","[nah, dont, think, goe, usf, live, around, tho...","[nah, dont, think, go, usf, life, around, though]"
...,...,...,...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,...","[2nd, time, tried, 2, contact, u, u, å£750, po...","[2nd, time, tri, 2, contact, u, u, å£750, poun...","[2nd, time, tried, 2, contact, u, u, å£750, po..."
5568,0,Will Ì_ b going to esplanade fr home?,Will Ì b going to esplanade fr home,"[will, ì, b, going, to, esplanade, fr, home]","[ì, b, going, esplanade, fr, home]","[ì, b, go, esplanad, fr, home]","[ì, b, going, esplanade, fr, home]"
5569,0,"Pity, * was in mood for that. So...any other s...",Pity was in mood for that Soany other suggest...,"[pity, was, in, mood, for, that, soany, other,...","[pity, mood, soany, suggestions]","[piti, mood, soani, suggest]","[pity, mood, soany, suggestion]"
5570,0,The guy did some bitching but I acted like i'd...,The guy did some bitching but I acted like id ...,"[the, guy, did, some, bitching, but, i, acted,...","[guy, bitching, acted, like, id, interested, b...","[guy, bitch, act, like, id, interest, buy, som...","[guy, bitching, acted, like, id, interested, b..."


#### Rozdzielenie na X, y

In [5]:
X = spam_dataset['Lemmatized_Text'].apply(lambda x: ' '.join(x))
y = spam_dataset['Spam']

#### Rozdzielenie danych na zbiory treningowe oraz testowe

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#### Wektoryzacja TF-IDF

In [7]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
feature_names = tfidf.vocabulary_.keys()

In [8]:
clf_tfidf = RandomForestClassifier()
clf_tfidf.fit(X_train_tfidf, y_train)
print("Wartość Score dla RandomForestCalssifier, przy TfidfVectorizer:",clf_tfidf.score(X_test_tfidf, y_test))

Wartość Score dla RandomForestCalssifier, przy TfidfVectorizer: 0.9757847533632287


#### Wektoryzacja CountVectorizer

In [9]:
count = CountVectorizer()
X_train_count = count.fit_transform(X_train)
X_test_count = count.transform(X_test)
clf_count = RandomForestClassifier()
clf_count.fit(X_train_count, y_train)
print("Wartość Score dla RandomForestCalssifier, przy CountVectorizer:",clf_count.score(X_test_count, y_test))

Wartość Score dla RandomForestCalssifier, przy CountVectorizer: 0.9739910313901345


#### Określenie które słowa są najwazniejsze oraz określenie ich wazności

In [10]:
importances = clf_tfidf.feature_importances_
feature_imp_df = pd.DataFrame({'Word': feature_names, 'Word Importance': importances}).sort_values('Word Importance', ascending=False) 
print(feature_imp_df)

            Word  Word Importance
1661     garbage         0.031545
7093       32323         0.029733
4593       board         0.025915
2994        arul         0.021489
1889       ardìä         0.019107
...          ...              ...
4850     busetop         0.000000
4851    knowthis         0.000000
4852  tirunelvai         0.000000
4853     plumber         0.000000
3910        jsut         0.000000

[7820 rows x 2 columns]


#### Stworzenie listy z słów których wazność jest większa niż 0.001

In [11]:
imp_word_lis = feature_imp_df[feature_imp_df['Word Importance']>0.001].iloc[:,0].to_list()
imp_word_lis

['garbage',
 '32323',
 'board',
 'arul',
 'ardìä',
 'knickers',
 'ukmobiledate',
 'improve',
 'distract',
 'nyc',
 '3750',
 'completed',
 'carton',
 'tape',
 'later',
 'sac',
 'request',
 'filthy',
 'currently',
 'treasure',
 'loved',
 'supose',
 'hp',
 'movie',
 'league',
 'havin',
 'tncs',
 'small',
 '7mahaveer',
 'bitch',
 'watching',
 'youany',
 'kintu',
 'parish',
 'ambitious',
 'dying',
 'jamz',
 'topped',
 'messageit',
 'either',
 'vodka',
 '89034',
 '500000',
 'win',
 'wwwapplausestorecom',
 'send',
 'cheetos',
 '3mobile',
 'starving',
 'castor',
 'dodgey',
 'lunchyou',
 'oyea',
 'somewhere',
 'tmorrowpls',
 'angel',
 'considering',
 'entropication',
 'din',
 'okie',
 'course',
 'jenny',
 'favor',
 'nokia6600',
 '140ppm',
 'anywhere',
 'complimentary',
 'passion',
 'kyou',
 'av',
 'deari',
 'high',
 'paining',
 'updat',
 'honey',
 'rinu',
 'ranju',
 'religiously',
 'scotland',
 'babe',
 'nag',
 'max6month',
 'ill',
 'grave',
 'blueu',
 'fat',
 'ive',
 'gsex',
 'leaf',
 'drastic

In [12]:
tfidf_imp =  TfidfVectorizer(min_df = 0.001)
X_train_imp = tfidf_imp.fit_transform(X_train)
X_test_imp = tfidf_imp.transform(X_test)

In [13]:
random_forest = RandomForestClassifier( n_jobs=-1)
params_rf = {'max_depth': [ None, 15],
             'min_samples_leaf': [1, 3],
             'n_estimators': [ 50, 100, 150 ],
             'min_samples_split': [2, 3, 5]}

rf_gridsearch = GridSearchCV(random_forest,
                             params_rf,
                             scoring='f1_micro',
                             cv=5,
                             verbose=10, n_jobs=-1)
rf_gridsearch.fit(X_train_imp, y_train)
print('\nBest hyperparameter:', rf_gridsearch.best_params_)
model = rf_gridsearch.best_estimator_
model.fit(X_train_imp, y_train)
print("Grid search for RandomForestClassifier Score Test (TfidfVectorizer):", model.score(X_test_imp, y_test))

Fitting 5 folds for each of 36 candidates, totalling 180 fits

Best hyperparameter: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 150}
Grid search for RandomForestClassifier Score Test (TfidfVectorizer): 0.9802690582959641


In [14]:
count_imp =  CountVectorizer(min_df = 0.001)
X_train_imp = tfidf_imp.fit_transform(X_train)
X_test_imp = tfidf_imp.transform(X_test)

In [15]:
random_forest = RandomForestClassifier( n_jobs=-1)
params_rf = {'max_depth': [ None, 15 ],
             'min_samples_leaf': [1, 3],
             'n_estimators': [ 50, 100, 150 ],
             'min_samples_split': [2, 3, 5]}

rf_gridsearch = GridSearchCV(random_forest,
                             params_rf,
                             scoring='f1_micro',
                             cv=5,
                             verbose=10, n_jobs=-1)
rf_gridsearch.fit(X_train_imp, y_train)
print('\nBest hyperparameter:', rf_gridsearch.best_params_)
model = rf_gridsearch.best_estimator_
model.fit(X_train_imp, y_train)
print("Grid search for RandomForestClassifier Score Test (CountVectorizer):", model.score(X_test_imp, y_test))

Fitting 5 folds for each of 36 candidates, totalling 180 fits

Best hyperparameter: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
Grid search for RandomForestClassifier Score Test (CountVectorizer): 0.979372197309417


#### Udało się poprwić wynik Score zarowno dla CountVectorizer jak i dla TfidfVectorizer